# Classify the pics into different stages and download it in zip file. (16 Apr)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os
from google.colab import files
import shutil

# Define model constants
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/DSP/22 Apr/keras_model.h5"
LABEL_PATH = "/content/drive/MyDrive/Colab Notebooks/DSP/22 Apr/labels.txt"
INPUT_IMAGE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/DSP/22 Apr/input"
OUTPUT_IMAGE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/DSP/22 Apr/output"
NOT_SURE_CLASS = "NotSure"
NOT_SURE_THRESHOLD = 0.4

# Load the model
def load_keras_model(model_path):
    return load_model(model_path, compile=False)

# Load the labels
def load_class_names(label_path):
    return open(label_path, "r").readlines()

# Preprocess an image
def preprocess_image(image_path):
    size = (224, 224)
    # Replace this with the path to your image
    image = Image.open(image_path).convert("RGB")
    # resizing the image to be at least 224x224 and then cropping from the center
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    # turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    return normalized_image_array

# Predict the class of an image
def predict_class(model, data, class_names, not_sure_threshold, not_sure_class):
    prediction = model.predict(data)
    index = np.argmax(prediction)
    confidence_score = prediction[0][index]
    class_name = (
        not_sure_class
        if confidence_score < not_sure_threshold
        else class_names[index][2:-1]
    )
    return class_name, confidence_score

# Organize and move images
def organize_images(input_folder, output_folder, class_name, filename, processed_images, total_images, confidence_score):
    output_folder = os.path.join(output_folder, class_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Create the new filename with the required details
    new_filename = f"{processed_images} of {total_images}_{class_name}_{confidence_score:.4f}{os.path.splitext(filename)[1]}"
    new_path = os.path.join(output_folder, new_filename)

    os.rename(os.path.join(input_folder, filename), new_path)

# Main function
def main():
    model = load_keras_model(MODEL_PATH)
    class_names = load_class_names(LABEL_PATH)
    total_images = len(os.listdir(INPUT_IMAGE_FOLDER))
    processed_images = 0

    for filename in os.listdir(INPUT_IMAGE_FOLDER):
        filename_lower = filename.lower()

        if not (
            filename_lower.endswith(".jpg")
            or filename_lower.endswith(".jpeg")
            or filename_lower.endswith(".png")
        ):
            print(f"Skipping: {filename}")
            continue

        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        image_path = os.path.join(INPUT_IMAGE_FOLDER, filename)
        normalized_image_array = preprocess_image(image_path)

        data[0] = normalized_image_array

        class_name, confidence_score = predict_class(
            model, data, class_names, NOT_SURE_THRESHOLD, NOT_SURE_CLASS
        )

        processed_images += 1
        print(f"Image {processed_images}/{total_images}: {filename}   --->  class: {class_name},      score: {confidence_score}")

        organize_images(INPUT_IMAGE_FOLDER, OUTPUT_IMAGE_FOLDER, class_name, filename, processed_images, total_images, confidence_score)

    # Once output is generated, download the OUTPUT_IMAGE_FOLDER to local drive
    shutil.make_archive("/content/output_images", 'zip', OUTPUT_IMAGE_FOLDER)
    files.download("/content/output_images.zip")

if __name__ == "__main__":
    main()


1/1 [==============================] - 2s 2s/step
Image 1/30: IMG_1141.JPEG   --->  class: Unripe,      score: 0.9995187520980835
1/1 [==============================] - 0s 67ms/step
Image 2/30: TestRipe(15).jpg   --->  class: Ripe,      score: 0.6305958032608032
1/1 [==============================] - 0s 74ms/step
Image 3/30: TestOverripe(5).jpg   --->  class: Overripe,      score: 0.6750116348266602
1/1 [==============================] - 0s 46ms/step
Image 4/30: TestOverripe(4).jpg   --->  class: Overripe,      score: 0.8232879042625427
1/1 [==============================] - 0s 41ms/step
Image 5/30: TestRipe(41).jpg   --->  class: Ripe,      score: 0.9996308088302612
1/1 [==============================] - 0s 41ms/step
Image 6/30: TestRipe(40).jpg   --->  class: Ripe,      score: 0.9908457398414612
1/1 [==============================] - 0s 45ms/step
Image 7/30: WhatsApp Image 2024-05-13 at 11.00.39_33dfc26d.jpg   --->  class: Ripe,      score: 0.9995604157447815
1/1 [===================

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model

# Define model constants
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/DSP/22 Apr/keras_model.h5"

# Load the model
model = load_model(MODEL_PATH, compile=False)

# Print the model summary
model.summary()

Model: "sequential_72"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_69 (Sequential)  (None, 1280)              410208    
                                                                 
 sequential_71 (Sequential)  (None, 3)                 128400    
                                                                 
Total params: 538608 (2.05 MB)
Trainable params: 524528 (2.00 MB)
Non-trainable params: 14080 (55.00 KB)
_________________________________________________________________


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Define the confusion matrix
confusion_matrix = np.array([[80, 2, 1],
                             [1, 44, 0],
                             [0, 0, 72]])

# Create the true and predicted labels based on the confusion matrix
true_labels = []
pred_labels = []

for i, row in enumerate(confusion_matrix):
    for j, count in enumerate(row):
        true_labels.extend([i] * count)
        pred_labels.extend([j] * count)

# Define the target class labels
target_names = ['Ripe', 'Unripe', 'Overripe']

# Compute the classification report
report = classification_report(true_labels, pred_labels, target_names=target_names)

print(report)


              precision    recall  f1-score   support

        Ripe       0.99      0.96      0.98        83
      Unripe       0.96      0.98      0.97        45
    Overripe       0.99      1.00      0.99        72

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200

